<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/generateCode-codeLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peft
!pip install datasets
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.8 MB/s eta 0:00:00


In [ ]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset

In [ ]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
### 4bit quant ###

# model_path = "meta-llama/Llama-3.1-8B-Instruct"

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Quantization configuration
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# # Load Tokenizer and Model
# tokenizer = AutoTokenizer.from_pretrained(
#     model_path,
#     model_max_length=512,
#     padding_side="left",
#     add_eos_token=True,
# )
# tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     quantization_config=bnb_config,
#     # device_map="auto",
#     # trust_remote_code=True,
#     # use_auth_token=True,
# )

# model.eval()

In [ ]:
### Load Model Normaly ###

model_path = "meta-llama/Llama-3.1-8B-Instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

# Load Model without quantization
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # GPU 메모리 절약을 위해 float16 사용 (필요시 float32로 변경 가능)
    device_map="auto",  # 모델을 자동으로 GPU로 로드
)

model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [ ]:
dataset = load_dataset("donghuna/StaQC-similarQuestion")
dataset

README.md:   0%|          | 0.00/437 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9962 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'snippet', 'similarQuestion', 'Confidence'],
        num_rows: 9962
    })
})

In [ ]:
from tqdm import tqdm
import csv

csv_file = "generated_plan0-3000.csv"

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

start_row = 0
max_snippet_length = 1024
sliced_dataset = dataset['train'].select(range(10, 11))
last_prompt = ""


with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["question_id", "question", "snippet", "similarQuestion", "confidence", "plan"])

    for row in tqdm(sliced_dataset, desc="Generating plans", unit="sample"):
        question_id = row['question_id']
        question = row['question']
        snippet = row['snippet']
        similarQuestion = row['similarQuestion']
        confidence = row['Confidence']

        if len(snippet) > max_snippet_length:
            snippet = snippet[:max_snippet_length]

        prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Given a competitive programming problem generate a concrete planning to solve the problem.
The problem is provided along with a Python code snippet that can be used as a reference to help solve the problem.
Please generalize the provided code so that it can be applied to similar problems, and summarize the plan in 6 lines or less.
Focusing only on the key actions needed to construct the code in text form, not on generating the code itself.
Importnat: you should give only the planning to solve the problem. Do not add extra explanation or words.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
### Original Question :
{question}
### Code Snippet :
{snippet}
### Planning to solve the problem in 6 lines or less:<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

        # 입력 토큰의 길이 계산
        input_length = inputs['input_ids'].shape[1]

        # 입력 프롬프트 토큰 수에 128을 더한 max_length 설정
        max_length = input_length + 256

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=max_length,
                eos_token_id=terminators,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.2
            )

            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

            generated_plan = ""

            keyword = "<|start_header_id|>assistant<|end_header_id|>"
            if keyword in generated_text:
                generated_plan = generated_text.split(keyword)[1].strip()

            keyword = "<|eot_id|>"
            if keyword in generated_plan:
                generated_plan = generated_plan.split(keyword)[0].strip()

            if not generated_plan:
                continue

            # print(generated_plan)
            writer.writerow([question_id, question, snippet, similarQuestion, confidence, generated_plan])



Generating plans: 100%|██████████| 1/1 [00:05<00:00,  5.19s/sample]

1. Define the table structure using SQLAlchemy's `Table` class.
2. Create a property to generate the query string from the list of table names.
3. Split the string of table names into a list.
4. Construct the raw SQL query using a generator expression and `UNION ALL`.
5. Execute the query using the SQLAlchemy session.
6. Fetch and return the results.


In [ ]:
!cp generated_plan0-3000.csv /content/drive/MyDrive/


In [ ]:
import pandas as pd

df = pd.read_csv('generated_plan.csv')
print(df['question'][0])
print(df['plan'][0])

In [ ]:
print(df.columns)
print(df.shape)

In [ ]:
# from datasets import Dataset
# from google.colab import userdata

# plan_dataset = Dataset.from_pandas(df)
# plan_dataset.push_to_hub("donghuna/StaQC-plan", token=userdata.get('HUG_TOKEN'))